In [4]:
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import time
import requests

In [5]:
thresholds = {
    '1' : 8,
    '2' : 8,
    '3' : 11,
    '4' : 10

}
telegramMessage = {
    '1': 'Female toilet: \nCubicle 4\nToilet paper dispenser is empty',
    '2': 'Female toilet: \nCubicle 4\nToilet paper dispenser is empty',
    '3': 'Female toilet: \nTrash bin is full',
    '4': 'Female toilet: \nCubicle 4\nSanitary bin is full',
}


In [3]:

#set Telegram channel Url
telegramURL = "https://api.telegram.org/bot733055974:AAEsyKoq3z5mquv0k_xmzugu_dej_1MbdtA/sendMessage?chat_id=-1001267215209&parse_mode=Markdown&text="


#set datebase configuration Code
database_settings = { "username": "admin",
                     "password": "abhyuday98",
                     "ip_address":"iotg5.ccspnys1uzca.us-east-1.rds.amazonaws.com",
                     "port": "3306",
                     "database_name": "iotg5"
                            }
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(database_settings['username'],
                                                                              database_settings['password'],
                                                                              database_settings['ip_address'],
                                                                              database_settings['port'],
                                                                              database_settings['database_name']
                                                                              ))
# Time
setTimeThreshold = 15

In [ ]:
while True:
    with engine.connect() as con:
        statement = text("call dbInactivityOverMinutes({})".format(setTimeThreshold))
        resultArray = con.execute(statement)
        for row in resultArray:
            print(row['tpCount'])
#             firstAndLastRecordOverXMinutesForToiletPaper
            if float(row['tpCount'])>0:
                statement = text("call firstAndLastRecordOverXMinutesForToiletPaper({})".format(setTimeThreshold))
                tpArrayArray = con.execute(statement)
                sendTopupUpdateOnTelegramForToiletPaper(tpArrayArray)
            else:
                sendTelegramMessage("Female toilet: \nCubicle 4\nToilet paper dispenser \nSensor is down")
            if float(row['bin1Count'])>0:
                statement = text("call firstAndLastRecordOverXMinutesForLitterBin1({})".format(setTimeThreshold))
                tpArrayArray = con.execute(statement)
                sendTopupUpdateOnTelegramForLitterBin(tpArrayArray,'3')
            else:
                sendTelegramMessage( "Female toilet: \nTrash bin \nSensor is down")
            if float(row['bin2Count'])>0:
                statement = text("call firstAndLastRecordOverXMinutesForLitterBin2({})".format(setTimeThreshold))
                tpArrayArray = con.execute(statement)
                sendTopupUpdateOnTelegramForLitterBin(tpArrayArray,'4')
            else:
                sendTelegramMessage("Female toilet: \nCubicle 4\nSanitary bin \nSensor is down")
            
            print(row['bin1Count'])
            print(row['bin2Count'])


    time.sleep(900)

62
54
60
